In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import DeepMed_bin
import DeepMed_bin_cv
import DeepMed_cont
import DeepMed_cont_cv
import DeepMed_cv
import warnings
warnings.filterwarnings("ignore") 

In [6]:
import pyreadr
result = pyreadr.read_r('../../DeepMed-main/data/example.RData')
print(result.keys())
x=np.array(result['x'])
y=np.array(result['y'])
d=np.array(result['d'])
m=np.array(result['m'])

odict_keys(['x', 'y', 'd', 'm'])


In [7]:
def DeepMed(y,d,m,x,method="DNN",hyper_grid=None,epochs=5,batch_size=100,trim=0.05):
    if method !="Lasso":
        hyper_grid = pd.DataFrame(hyper_grid)
        hyper=DeepMed_cv(y,d,m,x,method,hyper_grid,epochs,batch_size)
    else:
        hyper = np.empty(shape = (2,30))
        
    mbin = 1*((len(np.unique(m))==2) & (min(m)==0) & (max(m)==1))
    if mbin==0:
              temp=DeepMed_cont.DeepMed_cont(y,d,m,x,method,hyper,trim)
    else:
              temp=DeepMed_cont.DeepMed_bin(y,d,m,x,method,hyper,trim)
    ATE = np.array(temp)
    eff=ATE[0:5]
    se=np.sqrt(ATE[5:10]/ATE[10])
    results = [eff,se,2*norm.cdf(-abs(eff/se),loc=0,scale=1)]
    results = pd.DataFrame(results)
    #results= pd.concat([pd.Series(eff), pd.Series(se), pd.Series(2*norm.cdf(-abs(eff/se),loc=0,scale=1))])
    results.columns=["total", "dir.treat", "dir.control", "indir.treat", "indir.control"]
    results.index=["effect","se","pval"]
    ntrimmed=len(d)-ATE[10]
                       
    return results,ntrimmed


In [8]:
DeepMed(y,d,m,x,method="Lasso")

(               total     dir.treat   dir.control  indir.treat  indir.control
 effect  1.031567e+00  4.142282e-01  9.661725e-01     0.065394   6.173385e-01
 se      1.212650e-01  6.350891e-02  1.082719e-01     0.026053   9.424911e-02
 pval    1.789296e-17  6.920821e-11  4.514566e-19     0.012072   5.750921e-11,
 0.0)